In [2]:
%matplotlib widget
%matplotlib inline
%matplotlib notebook
%reset -f

In [3]:
import matplotlib.pyplot
import matplotlib.pyplot as plt
import numpy as np
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.patches as patches
#import sympy as sy
import scipy.constants as consp
from time import time
from joblib import Parallel, delayed
from PIL import Image
from time import time
from matplotlib import cm
import os
import numpy.ma as ma
import math
from joblib import Memory

#cachedir = 'chach'
#memory = Memory(cachedir, verbose=0)


def Bernoli(x,y,nei,c):
    z=(x+mparam[7,nei]+np.log(mparam[0,nei]))-(y+mparam[7,c]+np.log(mparam[0,c]))
    if abs(z)<1e-5:
        B=1 - .5 * z + 1/12.*(z)**2
    else:
        B=z/(np.exp(z)-1)
    return B

def mume(x,y):
    mu=2*(mparam[4,x]*mparam[4,y])/(mparam[4,x]+mparam[4,y])
    return mu

def Bernolip(x,y,nei,c):
    z=(y+mparam[7,c]+mparam[2,c]-np.log(mparam[1,c]))-(x+mparam[7,nei]+mparam[2,nei]-np.log(mparam[1,nei]))
    if abs(z)<1e-5:
        B=1 - .5 * z + 1/12.*(z)**2
    else:
        B=z/(np.exp(z)-1)
    return B

def mump(x,y):
    mu=2*(mparam[5,x]*mparam[5,y])/(mparam[5,x]+mparam[5,y])
    return mu

#####################################
    
T=300
mu_0=1e-4
N_0=1.e25
V_t=consp.k*T/consp.e
length=np.sqrt(consp.epsilon_0 * V_t / (consp.e * N_0))
St=length**2 / (mu_0*V_t)

############################################

def load_raw2(filename,width,height,S,N):
    of=width*height*4*(S-1)
    with open(filename,'rb') as raw:
        raw.seek(of)
        Im=np.fromfile(raw,np.dtype('uint8'),width*height*N).reshape(height, width)
    #raw.close
    return Im

#ImXBIC=np.array(Image.open('xbic.tif'))
#Tra=np.where(ImXBIC==1)
#ImXBIC[Tra]=0

ImXBIC=np.load('ImXBICS_raw.npy')

IM=load_raw2("Geom_02_139_25.raw",139,25,1,1)
CZTS=np.where(IM==186)
CdS=np.where(IM==155)
ZnO=np.where(IM==124)
maila=np.zeros(IM.shape)
maila[CZTS]=1
maila[CdS]=2
maila[ZnO]=3
#mail=mailabed[:,1:50]
mail=maila[:,:]
J,I=np.where(mail>0)
npy,npx=mail.shape
m1,n1=len(J),len(I)
pos=np.stack((J, I), axis=-1)
mtype=mail[J,I]

fig, axes = plt.subplots(figsize=(10, 10))
plt.imshow((mail),cmap='RdBu',vmin=-3,vmax=3)
plt.show()


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [4]:
m1

1435

In [5]:
mt1 = {'Nc': 2.2e24, 'Nv': 1.8e25, 'Eg': 1.4, 'epsilon': 10,  'mu_e': 100e-4, 'mu_h': 25e-4, 'tau_e': 2e-8,
              'affinity': 4.7, 'tau_h': 1e-5,'V':0,'Sn':1e7*1e-2,'Sp':1e7*1e-2,'work f':5.6}
mt2 = {'Nc': 2.2e24, 'Nv': 1.8e25, 'Eg': 2.4, 'epsilon': 10,  'mu_e': 100e-4, 'mu_h': 25e-4, 'tau_e': 1e-12,
              'affinity': 4.5, 'tau_h': 5e-13,'V':0,'Sn':1e7*1e-2,'Sp':1e7*1e-2,'work f':5.3}
mt3 = {'Nc': 2.2e24, 'Nv': 1.8e25, 'Eg': 3.3, 'epsilon': 9,  'mu_e': 100e-4, 'mu_h': 25e-4, 'tau_e': 1e-12,
              'affinity': 4.5, 'tau_h': 1e-7,'V':0,'Sn':1e7*1e-2,'Sp':1e7*1e-2,'work f':4.6}
scale={'Nc': N_0, 'Nv': N_0, 'Eg': V_t, 'epsilon': 1,  'mu_e': mu_0, 'mu_h': mu_0, 'tau_e': St,
              'affinity': V_t, 'tau_h': St,'V':V_t,'Sn':length/St,'Sp':length/St,'work f':V_t}


Na1=3e22
Nd2=1e22
Nd3=1e23
#Nd3=10

mparam=np.zeros((len(mt1),m1))
doping=np.zeros((m1,1))
for i, key in enumerate( mt1):
    mparam[i,:]=np.where(mtype==1,mt1[key]/scale[key],np.where(mtype==2,mt2[key]/scale[key],np.where(mtype==3,mt3[key]/scale[key],0)))
    #print(i,[key])
Cm=np.where(mtype==1,-Na1/N_0,np.where(mtype==2,Nd2/N_0,np.where(mtype==3,Nd3/N_0,0)))
#print(Cm)
ni=(np.sqrt(mparam[0]*mparam[1])) * np.exp(-mparam[2]/2)
mparam[12,0],mparam[12,40],mparam[12,-1]

(205.0131534807181, 177.93594453043457, 216.61767160226816)

In [6]:
%%time
def load(folder):
    originpath=os.getcwd()
    Path = "/"+folder
    os.chdir(originpath+Path)
    filelist = sorted(os.listdir(originpath+Path))
    data=np. array([])
    for i in filelist: 
        if i.startswith('dose') :
            Data=np.loadtxt(i)
            row_idx = np.array(np.where(Data[:,0]==9.950000E-05)[0])
            col_idx = np.array([1, 2])
            Data=Data[row_idx[:, None], col_idx]
            if data.size==0:
                data=Data
            else:
                data=np.vstack((data,Data))
        print(i.title())
    os.chdir(originpath)
    return data

data=load('CZTS')
r1=data[16:90:28,1].reshape(3,1)
r2=data[99:200:3,1].reshape(34,1)
Gczts=np.vstack((r1,r2))

data=load('CdS')
r1=data[16:90:28,1].reshape(3,1)
r2=data[99:200:3,1].reshape(34,1)
Gcds=np.vstack((r1,r2))

data=load('ZnO')
r1=data[16:90:28,1].reshape(3,1)
r2=data[99:200:3,1].reshape(34,1)
Gzno=np.vstack((r1,r2))

Gall=np.hstack((Gczts,Gcds,Gzno))

G=np.zeros((m1,m1))

for p in range(m1):
    if mtype[p]==1:
        a=np.array((I[p],J[p]))
        dis=math.floor(np.linalg.norm(np.array((npx-1,J[p]))-a))
        disl=math.floor(np.linalg.norm(np.array((0,J[p]))-a))
        if (dis<17):
            for n in range(m1):
                dis2=math.floor(np.linalg.norm(np.array((I[n],J[n]))-np.array((npx-1,J[n]))))
                if ((mtype[n]==1) & (dis2>2*dis) & (math.floor(np.linalg.norm(np.array((I[n],J[n])-a)))<17)):
                    G[p,n]=(Gall[math.floor(np.linalg.norm(np.array((I[n],J[n]))-a)),0]*4.56*1e12*1e6)/(3*1.5)*(length**2)/(V_t*N_0*mu_0)
                    mer=np.array(np.where((J==J[n])&(I==(npx-1-I[n]-(2*dis+1))))[0])
                    if (mer.size>0):
                        if (mtype[mer[0]]==1):
                            G[p,mer[0]]=G[p,n]
                elif((mtype[n]==1) & (math.floor(np.linalg.norm(np.array((I[n],J[n])-a)))<17)):
                    G[p,n]=(Gall[math.floor(np.linalg.norm(np.array((I[n],J[n]))-a)),0]*4.56*1e12*1e6)/(3*1.5)*(length**2)/(V_t*N_0*mu_0)
        elif(disl<17):
            for n in range(m1):
                dis2=math.floor(np.linalg.norm(np.array((I[n],J[n]))-np.array((0,J[n]))))
                if ((mtype[n]==1) & (dis2>2*disl) & (math.floor(np.linalg.norm(np.array((I[n],J[n])-a)))<17)):
                    G[p,n]=(Gall[math.floor(np.linalg.norm(np.array((I[n],J[n]))-a)),0]*4.56*1e12*1e6)/(3*1.5)*(length**2)/(V_t*N_0*mu_0)
                    mer=np.array(np.where((J==J[n])&(I==(npx-1-I[n]+(2*disl+1))))[0])
                    if (mer.size>0):
                        if (mtype[mer[0]]==1):
                            G[p,mer[0]]=G[p,n]
                elif((mtype[n]==1) & (math.floor(np.linalg.norm(np.array((I[n],J[n])-a)))<17)):
                    G[p,n]=(Gall[math.floor(np.linalg.norm(np.array((I[n],J[n]))-a)),0]*4.56*1e12*1e6)/(3*1.5)*(length**2)/(V_t*N_0*mu_0)   
        else:
            for n in range(m1):
                if ((mtype[n]==1) & (math.floor(np.linalg.norm(np.array((I[n],J[n])-a)))<37)):
                    G[p,n]=(Gall[math.floor(np.linalg.norm(np.array((I[n],J[n]))-a)),0]*4.56*1e12*1e6)/(3*1.5)*(length**2)/(V_t*N_0*mu_0)
    elif mtype[p]==2:
        a=np.array((I[p],J[p]))
        dis=math.floor(np.linalg.norm(np.array((npx-1,J[p]))-a))
        disl=math.floor(np.linalg.norm(np.array((0,J[p]))-a))
        if (dis<17):
            for n in range(m1):
                dis2=math.floor(np.linalg.norm(np.array((I[n],J[n]))-np.array((npx-1,J[n]))))
                if ((mtype[n]==2) & (dis2>2*dis) & (math.floor(np.linalg.norm(np.array((I[n],J[n])-a)))<17)):
                    G[p,n]=(Gall[math.floor(np.linalg.norm(np.array((I[n],J[n]))-a)),1]*4.82*1e12*1e6)/(3*2.4)*(length**2)/(V_t*N_0*mu_0)
                    mer=np.array(np.where((J==J[n])&(I==(npx-1-I[n]-(2*dis+1))))[0])
                    if (mer.size>0):
                        if (mtype[mer[0]]==2):
                            G[p,mer[0]]=G[p,n]
                elif((mtype[n]==2) & (math.floor(np.linalg.norm(np.array((I[n],J[n])-a)))<17)):
                    G[p,n]=(Gall[math.floor(np.linalg.norm(np.array((I[n],J[n]))-a)),1]*4.82*1e12*1e6)/(3*2.4)*(length**2)/(V_t*N_0*mu_0)
        elif(disl<17):
            for n in range(m1):
                dis2=math.floor(np.linalg.norm(np.array((I[n],J[n]))-np.array((0,J[n]))))
                if ((mtype[n]==2) & (dis2>2*disl) & (math.floor(np.linalg.norm(np.array((I[n],J[n])-a)))<17)):
                    G[p,n]=(Gall[math.floor(np.linalg.norm(np.array((I[n],J[n]))-a)),1]*4.82*1e12*1e6)/(3*2.4)*(length**2)/(V_t*N_0*mu_0)
                    mer=np.array(np.where((J==J[n])&(I==(npx-1-I[n]+(2*disl+1))))[0])
                    if (mer.size>0):
                        if (mtype[mer[0]]==2):
                            G[p,mer[0]]=G[p,n]
                elif((mtype[n]==2) & (math.floor(np.linalg.norm(np.array((I[n],J[n])-a)))<17)):
                    G[p,n]=(Gall[math.floor(np.linalg.norm(np.array((I[n],J[n]))-a)),1]*4.82*1e12*1e6)/(3*2.4)*(length**2)/(V_t*N_0*mu_0)
            
        else:
            for n in range(m1):
                if ((mtype[n]==2) & (math.floor(np.linalg.norm(np.array((I[n],J[n])-a)))<37)):
                    G[p,n]=(Gall[math.floor(np.linalg.norm(np.array((I[n],J[n]))-a)),1]*4.82*1e12*1e6)/(3*2.4)*(length**2)/(V_t*N_0*mu_0)
    elif mtype[p]==3:
        a=np.array((I[p],J[p]))
        dis=math.floor(np.linalg.norm(np.array((npx-1,J[p]))-a))
        disl=math.floor(np.linalg.norm(np.array((0,J[p]))-a))
        if (dis<17):
            for n in range(m1):
                dis2=math.floor(np.linalg.norm(np.array((I[n],J[n]))-np.array((npx-1,J[n]))))
                if ((mtype[n]==3) & (dis2>2*dis) & (math.floor(np.linalg.norm(np.array((I[n],J[n])-a)))<17)):
                    G[p,n]=(Gall[math.floor(np.linalg.norm(np.array((I[n],J[n]))-a)),2]*5.61*1e12*1e6)/(3*3.3)*(length**2)/(V_t*N_0*mu_0)
                    mer=np.array(np.where((J==J[n])&(I==(npx-1-I[n]-(2*dis+1))))[0])
                    if (mer.size>0):
                        if (mtype[mer[0]]==3):
                            G[p,mer[0]]=G[p,n]
                elif((mtype[n]==3) & (math.floor(np.linalg.norm(np.array((I[n],J[n])-a)))<17)):
                    G[p,n]=(Gall[math.floor(np.linalg.norm(np.array((I[n],J[n]))-a)),2]*5.61*1e12*1e6)/(3*3.3)*(length**2)/(V_t*N_0*mu_0)
        elif(disl<17):
            for n in range(m1):
                dis2=math.floor(np.linalg.norm(np.array((I[n],J[n]))-np.array((0,J[n]))))
                if ((mtype[n]==3) & (dis2>2*disl) & (math.floor(np.linalg.norm(np.array((I[n],J[n])-a)))<17)):
                    G[p,n]=(Gall[math.floor(np.linalg.norm(np.array((I[n],J[n]))-a)),2]*5.61*1e12*1e6)/(3*3.3)*(length**2)/(V_t*N_0*mu_0)
                    mer=np.array(np.where((J==J[n])&(I==(npx-1-I[n]+(2*disl+1))))[0])
                    if (mer.size>0):
                        if (mtype[mer[0]]==3):
                            G[p,mer[0]]=G[p,n]
                elif((mtype[n]==3) & (math.floor(np.linalg.norm(np.array((I[n],J[n])-a)))<17)):
                    G[p,n]=(Gall[math.floor(np.linalg.norm(np.array((I[n],J[n]))-a)),2]*5.61*1e12*1e6)/(3*3.3)*(length**2)/(V_t*N_0*mu_0)    
        else:
            for n in range(m1):
                if ((mtype[n]==3) & (math.floor(np.linalg.norm(np.array((I[n],J[n])-a)))<37)):
                    G[p,n]=(Gall[math.floor(np.linalg.norm(np.array((I[n],J[n]))-a)),2]*5.61*1e12*1e6)/(3*3.3)*(length**2)/(V_t*N_0*mu_0)
                  

Dose-Charge-02.Dat
Dose-Charge-03.Dat
Dose-Charge-04.Dat
Dose-Charge-05.Dat
Dose-Charge-06.Dat
Dose-Charge-02.Dat
Dose-Charge-03.Dat
Dose-Charge-04.Dat
Dose-Charge-05.Dat
Dose-Charge-06.Dat
Dose-Charge-02.Dat
Dose-Charge-03.Dat
Dose-Charge-04.Dat
Dose-Charge-05.Dat
Dose-Charge-06.Dat
CPU times: user 1min 4s, sys: 100 ms, total: 1min 4s
Wall time: 1min 12s


In [7]:
masked = np.ma.masked_where(mail == 0, mail)


As=np.zeros((npy,npx))
As[J,I]=G[115]
masked.data[masked.data!=0]=As[masked.data!=0]
plt.figure(figsize=(10, 10));plt.imshow(masked,cmap='jet')  

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [8]:
'''
r1=np.arange(95,125)
chang=[]
for item in(r1):
    chang.append(np.where((pos[:,1]==item) & (mtype[:]==2)))
'''

'\nr1=np.arange(95,125)\nchang=[]\nfor item in(r1):\n    chang.append(np.where((pos[:,1]==item) & (mtype[:]==2)))\n'

In [9]:
#chang

In [10]:
'''
chang[10][0]
for i in range(len(chang)):
    mparam[10,chang[i][0]]=0*1e-2/(length/St)
    mparam[11,chang[i][0]]=0*1e-2/(length/St)
'''    


'\nchang[10][0]\nfor i in range(len(chang)):\n    mparam[10,chang[i][0]]=0*1e-2/(length/St)\n    mparam[11,chang[i][0]]=0*1e-2/(length/St)\n'

In [11]:
mparam[10,9]


14.62083231461082

In [12]:
'''
r1=np.where(pos[:,1]==15)
r2=np.where(pos[:,1]==80)
r3=np.where(pos[:,1]==110)
'''

'\nr1=np.where(pos[:,1]==15)\nr2=np.where(pos[:,1]==80)\nr3=np.where(pos[:,1]==110)\n'

In [13]:
'''
G1=G[r1]
G2=G[r2]
G3=G[r3]
'''

'\nG1=G[r1]\nG2=G[r2]\nG3=G[r3]\n'

In [21]:

def Bernoli2(x,y,nei,c):
    z=(x+mparam[7,nei]+np.log(mparam[0,nei]))-(y+mparam[7,c]+np.log(mparam[0,c]))
    B=np.where(abs(z)<1e-5,1 - .5 * z + 1/12.*(z)**2,z/(np.exp(z)-1))
    return B

def Bernolip2(x,y,nei,c):
    z=(y+mparam[7,c]+mparam[2,c]-np.log(mparam[1,c]))-(x+mparam[7,nei]+mparam[2,nei]-np.log(mparam[1,nei]))
    B=np.where(abs(z)<1e-5,1 - .5 * z + 1/12.*(z)**2,z/(np.exp(z)-1))
    return B



Wpb=[]
Neig=np.zeros((m1,4))
alph=np.zeros((m1,4))
alphb=np.zeros((m1,4))

for i in range(0,n1):
    SS=np.array(np.where((J==pos[i,0]+1)& (I==pos[i,1])))
    if SS.size>0:
        Neig[i,0]=SS[0,0]
        alph[i,0]=2
    else:
        Neig[i,0]=i
        alphb[i,0]=2
        
    NN=np.array(np.where((J==pos[i,0]-1)& (I==pos[i,1])))
    if NN.size>0:
        Neig[i,2]=NN[0,0]
        alph[i,2]=2
    else:
        Neig[i,2]=i
        alphb[i,2]=2
        
    if (pos[i,1]==0):
        if (mail[pos[i,0],pos[i,1]+npx-1]!=0):
            Wper=np.array(np.where((J==pos[i,0])& (I==pos[i,1]+npx-1)))[0,0]
            Neig[i,1]=Wper
            alph[i,1]=2
        else:
            Neig[i,1]=i
            alphb[i,1]=2
            Wpb.append(i)
    elif pos[i-1,1]==pos[i,1]-1:
        Neig[i,1]=i-1
        alph[i,1]=2
    else:
        Neig[i,1]=i
        alphb[i,1]=2
        
for i in range(0,n1-1):        
    if pos[i,1]==npx-1:
        if (mail[pos[i,0],pos[i,1]-(npx-1)]!=0):
            Eper=np.array(np.where((J==pos[i,0])& (I==pos[i,1]-(npx-1))))[0,0]
            Neig[i,3]=Eper
            alph[i,3]=2
        else:
            Neig[i,3]=i
            alphb[i,3]=2
    elif (pos[i+1,1]==pos[i,1]+1):
        Neig[i,3]=i+1
        alph[i,3]=2
    else:
        Neig[i,3]=i
        alphb[i,3]=2
if pos[n1-1,1]==npx-1:
    if (mail[pos[n1-1,0],pos[n1-1,1]-(npx-1)]!=0):
        Eper=np.array(np.where((J==pos[n1-1,0])& (I==pos[n1-1,1]-(npx-1))))[0,0]
        Neig[n1-1,3]=Eper
        alph[n1-1,3]=2
    else:
        Neig[n1-1,3]=n1-1
        alphb[n1-1,3]=2
else:
        Neig[n1-1,3]=n1-1
        alphb[n1-1,3]=2 
Neig=Neig.astype("int")

psitt1=np.where(Cm>0,np.log((Cm/(mparam[0])))-mparam[7],-np.log((-Cm/mparam[1]))-mparam[7]-mparam[2])
psitt=-mparam[12]
ns=mparam[0]*np.exp((psitt+mparam[7]))
ps=mparam[1]*np.exp((-psitt-mparam[7]-mparam[2]))

d=5e-8
dx=d/length
dy=d/length

valN=alph*(mparam[3,Neig]*np.repeat(mparam[3].reshape(n1,1),4,axis=1))/(mparam[3,Neig]+np.repeat(mparam[3].reshape(n1,1),4,axis=1))
valB=alphb*mparam[3,Neig]
Gbtest=valB*psitt[Neig]

rows=np.arange(n1).reshape(n1,1)
Atest=np.zeros((m1,n1))

Atest[rows,Neig]=valN



psieq=psitt
nseq=ns
pseq=ps
psie=psitt

damp=1

from numpy.linalg import inv
from numpy.linalg import solve
for j in range(0,100):
    np.fill_diagonal(Atest,-((np.sum(valB+valN,axis=1))+dx*dy*(nseq+pseq)))
    Ftest=-(np.sum(Gbtest,axis=1))+dx*dy*(-(nseq+pseq)*psieq+nseq-pseq-Cm)
    psieq=solve((Atest),Ftest)
    psieq=damp*psieq+(1-damp)*psie
    nseq=mparam[0]*np.exp((psieq+mparam[7]))
    pseq=mparam[1]*np.exp((-psieq-mparam[7]-mparam[2]))
    aaa=np.max(abs(psie-psieq))
    psie=psieq
    print(aaa)

aa=np.where(I==0)[0]

x=np.linspace(0,4,npx+1)
#y=np.linspace(0,4e-6,aa.shape[0])
y=np.arange(aa.shape[0])
fig, ax = plt.subplots(figsize=(8, 8))
l1, = ax.plot(y*1e6, -V_t*(psieq[aa]+ mparam[7,aa]-np.log(nseq[aa]/mparam[0,aa])), lw=2, color='#2e89cf', ls=':')
l2, = ax.plot(y*1e6, -V_t*(psieq[aa] + mparam[7,aa] + mparam[2,aa]+np.log(pseq[aa]/mparam[1,aa])), lw=2, color='#cf392e', ls=':')
l3, = ax.plot(y*1e6, -V_t * (psieq[aa] + mparam[7,aa]), lw=2, color='k', ls='-')
l4, = ax.plot(y*1e6, -V_t * (psieq[aa] + mparam[7,aa] + mparam[2,aa]), lw=2, color='k', ls='--')
#l5, = ax.plot(y*1e6, -V_t * (psieq[aa] ), lw=2, color='b', ls='-.')

fig.legend([l3,l2,l4,], [r'$\mathregular{E_{C}}$',
                        r'$\mathregular{E_{F}}$',
                        r'$\mathregular{E_{V}}$',
                        r'$\mathregular{E_{ref}}$',
                       ])
ax.set_xlabel(r'Position [$\mathregular{\mu m}$]')
ax.set_ylabel('Energy [eV]')
plt.title('Band diagram Equilibrim')


/opt/conda/envs/python3/lib/python3.7/site-packages/ipykernel_launcher.py:79: RuntimeWarning: invalid value encountered in log


52.08871797134651
1.0000000000001137
1.0000000000001137
1.0000000000001137
1.0000000000001137
1.0000000000001137
1.0000000000001137
1.0000000000001137
1.0000000000001137
1.0000000000001137
1.0000000000001137
1.0000000000001137
1.0000000000000568
1.0000000000000853
1.0000000000000568
1.0000000000000853
1.0000000000000568
1.0000000000000284
1.0000000000000284
1.0
0.9999999999997726
0.9999999999994316
0.9999999999983231
0.9999999999955378
0.9999999999876366
0.9999999999664055
0.9999999999086242
0.9999999997512816
0.9999999993151789
0.9999999979874872
0.9999999923953169
0.9999999546678282
0.9999996292414437
0.9999967165380781
0.9999714853017565
0.9997652344748076
0.9982294889686614
0.9885488082683764
0.9418111062980188
0.7810602628542824
0.4337976365320344
0.09551190650915942
0.0035411221242611646
4.469507160820285e-06
6.906475391588174e-12
1.7053025658242404e-13
2.5579538487363607e-13
1.9895196601282805e-13
1.7053025658242404e-13
1.7053025658242404e-13
1.9895196601282805e-13
1.70530256582

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0.5, 1.0, 'Band diagram Equilibrim')

In [15]:
Sn=np.array((-1e7*1e-2,1e5*1e-2,1e7*1e-2))*St/length
Sp=np.array((1e7*1e-2,-1e5*1e-2,-1e7*1e-2))*St/length

In [16]:
Sn[2],len(mt1),mparam[12,-1],m1

(14.62083231461082, 13, 216.61767160226816, 1435)

In [22]:
# out_of
########### Initial guess Out of equilibrium
psic=psieq
nsc=nseq
psc=pseq
psie=pseq

####################### Boundary Conditions
ns,
ps,
psitt
mparam[9]

#############################################


PP=40
#alphbT=alphb/2
#alphbT[:,(2,3)]*=-1
#alphbT[Wpb,1]*=-1
#rows=np.arange(n1).reshape(n1,1)


def solve_out_2(mparam,Cm,ni,dx,dy,psieq,nseq,pseq,psitt,ns,ps,G,PP):
    psic=psieq
    nsc=nseq
    psc=pseq
    psie=pseq
    nse=nseq
    pse=pseq
    error=1
    kkk=0
    while error>=5e-5:
        kkk+=1
        ###############################################
        Antest=np.zeros((m1,n1))
        valuNn=(-alph/2)*mume(Neig,rows)*Bernoli2(psic[Neig],psic[rows],Neig,rows)
        valuBn=(-alphb*dx/2)*mparam[10,Neig]
        valuNnc=(alph/2)*mume(Neig,rows)*Bernoli2(psic[rows],psic[Neig],rows,Neig)
        valuBnc=-valuBn
        Antest[rows,Neig]=valuNn
        Rnc=(psc)/(mparam[8]*(nsc+ni)+mparam[6]*(psc+ni))
        np.fill_diagonal(Antest,((np.sum(valuBnc+valuNnc,axis=1))+Rnc*dx*dy))
        #####################################################################
        Gbntest=-valuBn*nseq[Neig]
        Rn=(-ni**2)/(mparam[8]*(nsc+ni)+mparam[6]*(psc+ni))
        
        Fntest=(np.sum(Gbntest,axis=1))+(G-Rn)*dx*dy
        #############################################################
        nsc=solve((Antest),Fntest)
        #nsc=damp*nsc+(1-damp)*nse
        #####################################################################
        Aptest=np.zeros((m1,n1))
        valuNp=(alph/2)*mump(Neig,rows)*Bernolip2(psic[Neig],psic[rows],Neig,rows)
        valuBp=(alphb*dx/2)*mparam[11,Neig]
        valuNpc=(-alph/2)*mump(Neig,rows)*Bernolip2(psic[rows],psic[Neig],rows,Neig)
        valuBpc=-valuBp
        Aptest[rows,Neig]=valuNp
        Rpc=(nsc)/(mparam[8]*(nsc+ni)+mparam[6]*(psc+ni))
        np.fill_diagonal(Aptest,((np.sum(valuBpc+valuNpc,axis=1))-Rpc*dx*dy))
        ##################################################################
        Gbptest=-valuBp*pseq[Neig]
        Rp=(-ni**2)/(mparam[8]*(nsc+ni)+mparam[6]*(psc+ni))
        Fptest=(np.sum(Gbptest,axis=1))-(G-Rp)*dx*dy
        #################################################################    
        psc=solve((Aptest),Fptest)
        #psc=damp*psc+(1-damp)*pse
        ##################################################################
        Atest=np.zeros((m1,n1))
        Atest[rows,Neig]=valN
        Gbtest=valB*(psitt[Neig]+mparam[9,Neig])
        np.fill_diagonal(Atest,-((np.sum(valB+valN,axis=1))+dx*dy*(nsc+psc)))
        Ftest=-(np.sum(Gbtest,axis=1))+dx*dy*(-(nsc+psc)*psic+nsc-psc-Cm)
        psic=solve((Atest),Ftest)
        psic=damp*psic+(1-damp)*psie
        #####################################################
        error=np.max(abs(psic-psie))
        print(kkk,error)
        psie=psic
        nse=nsc
        pse=psc
        ###############################################################
    JtL=0
    JtU=0
    ##################################################################
    jny=np.zeros((m1))
    jpy=np.zeros((m1))
    for i in range(0,n1):
        SS=np.array(np.where((J==pos[i,0]+1)& (I==pos[i,1])))
        if SS.size>0:
            neiS=SS[0,0]
            jny[i]=-1/dy*mume(neiS,i)*(Bernoli(psic[i],psic[neiS],i,neiS)*nsc[i]-Bernoli(psic[neiS],psic[i],neiS,i)*nsc[neiS])
            jpy[i]=1/dy*mump(neiS,i)*(Bernolip(psic[i],psic[neiS],i,neiS)*psc[i]-Bernolip(psic[neiS],psic[i],neiS,i)*psc[neiS])
    Jy=jny+jpy
    SBBm=np.where(J==int((J[0]+J[-1])/2))
    Jtsm=np.sum(Jy[SBBm])*dx
    pp=np.where(I==PP)[0]
    EFn=-V_t*(psic[pp]+ mparam[7,pp]-np.log(nsc[pp]/mparam[0,pp]))
    EFp=-V_t*(psic[pp] + mparam[7,pp] + mparam[2,pp]+np.log(psc[pp]/mparam[1,pp]))
    EC=-V_t * (psic[pp] + mparam[7,pp])
    EV=-V_t * (psic[pp] + mparam[7,pp] + mparam[2,pp])
    return JtL,JtU,psic,nsc,psc,EFn,EFp,EC,EV,Rn,Rp,Jtsm,error,Fptest,Jy,SBBm

In [18]:
int((J[0]+J[-1])/2)
5e-5>9e-5

False

In [19]:
0.01/2

0.005

In [28]:

%%time
resu=[]
if __name__ == '__main__':
    resu = Parallel(n_jobs=-1, verbose=20)(delayed(solve_out_2)(mparam,Cm,ni,dx,dy,psieq,nseq,pseq,psitt,ns,ps,g,40) for g in G)


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:   22.5s
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:   23.4s
[Parallel(n_jobs=-1)]: Done   3 tasks      | elapsed:   23.6s
[Parallel(n_jobs=-1)]: Done   4 tasks      | elapsed:   23.8s
[Parallel(n_jobs=-1)]: Done   5 tasks      | elapsed:   23.9s
[Parallel(n_jobs=-1)]: Done   6 tasks      | elapsed:   24.0s
[Parallel(n_jobs=-1)]: Done   7 tasks      | elapsed:   24.7s
[Parallel(n_jobs=-1)]: Done   8 tasks      | elapsed:   24.8s
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:   45.6s
[Parallel(n_jobs=-1)]: Done  10 tasks      | elapsed:   46.2s
[Parallel(n_jobs=-1)]: Done  11 tasks      | elapsed:   46.3s
[Parallel(n_jobs=-1)]: Done  12 tasks      | elapsed:   46.3s
[Parallel(n_jobs=-1)]: Done  13 tasks      | elapsed:   47.7s
[Parallel(n_jobs=-1)]: Done  14 tasks      | elapsed:   47.8s
[Parallel(n_jobs=-1)]: Done  15 tasks      | elapsed:   

[Parallel(n_jobs=-1)]: Done 132 tasks      | elapsed:  6.4min
[Parallel(n_jobs=-1)]: Done 133 tasks      | elapsed:  6.4min
[Parallel(n_jobs=-1)]: Done 134 tasks      | elapsed:  6.5min
[Parallel(n_jobs=-1)]: Done 135 tasks      | elapsed:  6.5min
[Parallel(n_jobs=-1)]: Done 136 tasks      | elapsed:  6.5min
[Parallel(n_jobs=-1)]: Done 137 tasks      | elapsed:  6.6min
[Parallel(n_jobs=-1)]: Done 138 tasks      | elapsed:  6.7min
[Parallel(n_jobs=-1)]: Done 139 tasks      | elapsed:  6.7min
[Parallel(n_jobs=-1)]: Done 140 tasks      | elapsed:  6.8min
[Parallel(n_jobs=-1)]: Done 141 tasks      | elapsed:  6.8min
[Parallel(n_jobs=-1)]: Done 142 tasks      | elapsed:  6.8min
[Parallel(n_jobs=-1)]: Done 143 tasks      | elapsed:  6.8min
[Parallel(n_jobs=-1)]: Done 144 tasks      | elapsed:  6.8min
[Parallel(n_jobs=-1)]: Done 145 tasks      | elapsed:  6.9min
[Parallel(n_jobs=-1)]: Done 146 tasks      | elapsed:  7.0min
[Parallel(n_jobs=-1)]: Done 147 tasks      | elapsed:  7.0min
[Paralle

[Parallel(n_jobs=-1)]: Done 265 tasks      | elapsed: 12.3min
[Parallel(n_jobs=-1)]: Done 266 tasks      | elapsed: 12.3min
[Parallel(n_jobs=-1)]: Done 267 tasks      | elapsed: 12.3min
[Parallel(n_jobs=-1)]: Done 268 tasks      | elapsed: 12.3min
[Parallel(n_jobs=-1)]: Done 269 tasks      | elapsed: 12.4min
[Parallel(n_jobs=-1)]: Done 270 tasks      | elapsed: 12.4min
[Parallel(n_jobs=-1)]: Done 271 tasks      | elapsed: 12.4min
[Parallel(n_jobs=-1)]: Done 272 tasks      | elapsed: 12.4min
[Parallel(n_jobs=-1)]: Done 273 tasks      | elapsed: 12.5min
[Parallel(n_jobs=-1)]: Done 274 tasks      | elapsed: 12.6min
[Parallel(n_jobs=-1)]: Done 275 tasks      | elapsed: 12.6min
[Parallel(n_jobs=-1)]: Done 276 tasks      | elapsed: 12.6min
[Parallel(n_jobs=-1)]: Done 277 tasks      | elapsed: 12.7min
[Parallel(n_jobs=-1)]: Done 278 tasks      | elapsed: 12.7min
[Parallel(n_jobs=-1)]: Done 279 tasks      | elapsed: 12.7min
[Parallel(n_jobs=-1)]: Done 280 tasks      | elapsed: 12.7min
[Paralle

[Parallel(n_jobs=-1)]: Done 398 tasks      | elapsed: 18.2min
[Parallel(n_jobs=-1)]: Done 399 tasks      | elapsed: 18.2min
[Parallel(n_jobs=-1)]: Done 400 tasks      | elapsed: 18.3min
[Parallel(n_jobs=-1)]: Done 401 tasks      | elapsed: 18.4min
[Parallel(n_jobs=-1)]: Done 402 tasks      | elapsed: 18.4min
[Parallel(n_jobs=-1)]: Done 403 tasks      | elapsed: 18.4min
[Parallel(n_jobs=-1)]: Done 404 tasks      | elapsed: 18.4min
[Parallel(n_jobs=-1)]: Done 405 tasks      | elapsed: 18.5min
[Parallel(n_jobs=-1)]: Done 406 tasks      | elapsed: 18.5min
[Parallel(n_jobs=-1)]: Done 407 tasks      | elapsed: 18.5min
[Parallel(n_jobs=-1)]: Done 408 tasks      | elapsed: 18.5min
[Parallel(n_jobs=-1)]: Done 409 tasks      | elapsed: 18.6min
[Parallel(n_jobs=-1)]: Done 410 tasks      | elapsed: 18.6min
[Parallel(n_jobs=-1)]: Done 411 tasks      | elapsed: 18.7min
[Parallel(n_jobs=-1)]: Done 412 tasks      | elapsed: 18.7min
[Parallel(n_jobs=-1)]: Done 413 tasks      | elapsed: 18.8min
[Paralle

[Parallel(n_jobs=-1)]: Done 531 tasks      | elapsed: 24.7min
[Parallel(n_jobs=-1)]: Done 532 tasks      | elapsed: 24.7min
[Parallel(n_jobs=-1)]: Done 533 tasks      | elapsed: 24.8min
[Parallel(n_jobs=-1)]: Done 534 tasks      | elapsed: 24.8min
[Parallel(n_jobs=-1)]: Done 535 tasks      | elapsed: 24.9min
[Parallel(n_jobs=-1)]: Done 536 tasks      | elapsed: 25.0min
[Parallel(n_jobs=-1)]: Done 537 tasks      | elapsed: 25.1min
[Parallel(n_jobs=-1)]: Done 538 tasks      | elapsed: 25.1min
[Parallel(n_jobs=-1)]: Done 539 tasks      | elapsed: 25.1min
[Parallel(n_jobs=-1)]: Done 540 tasks      | elapsed: 25.1min
[Parallel(n_jobs=-1)]: Done 541 tasks      | elapsed: 25.1min
[Parallel(n_jobs=-1)]: Done 542 tasks      | elapsed: 25.2min
[Parallel(n_jobs=-1)]: Done 543 tasks      | elapsed: 25.2min
[Parallel(n_jobs=-1)]: Done 544 tasks      | elapsed: 25.3min
[Parallel(n_jobs=-1)]: Done 545 tasks      | elapsed: 25.3min
[Parallel(n_jobs=-1)]: Done 546 tasks      | elapsed: 25.4min
[Paralle

...

[Parallel(n_jobs=-1)]: Done 1062 tasks      | elapsed: 51.2min
[Parallel(n_jobs=-1)]: Done 1063 tasks      | elapsed: 51.3min
[Parallel(n_jobs=-1)]: Done 1064 tasks      | elapsed: 51.4min
[Parallel(n_jobs=-1)]: Done 1065 tasks      | elapsed: 51.4min
[Parallel(n_jobs=-1)]: Done 1066 tasks      | elapsed: 51.5min
[Parallel(n_jobs=-1)]: Done 1067 tasks      | elapsed: 51.5min
[Parallel(n_jobs=-1)]: Done 1068 tasks      | elapsed: 51.6min
[Parallel(n_jobs=-1)]: Done 1069 tasks      | elapsed: 51.6min
[Parallel(n_jobs=-1)]: Done 1070 tasks      | elapsed: 51.7min
[Parallel(n_jobs=-1)]: Done 1071 tasks      | elapsed: 51.7min
[Parallel(n_jobs=-1)]: Done 1072 tasks      | elapsed: 51.8min
[Parallel(n_jobs=-1)]: Done 1073 tasks      | elapsed: 51.9min
[Parallel(n_jobs=-1)]: Done 1074 tasks      | elapsed: 51.9min
[Parallel(n_jobs=-1)]: Done 1075 tasks      | elapsed: 51.9min
[Parallel(n_jobs=-1)]: Done 1076 tasks      | elapsed: 52.0min
[Parallel(n_jobs=-1)]: Done 1077 tasks      | elapsed: 

[Parallel(n_jobs=-1)]: Done 1230 tasks      | elapsed: 59.3min
[Parallel(n_jobs=-1)]: Done 1231 tasks      | elapsed: 59.3min
[Parallel(n_jobs=-1)]: Done 1232 tasks      | elapsed: 59.4min
[Parallel(n_jobs=-1)]: Done 1233 tasks      | elapsed: 59.4min
[Parallel(n_jobs=-1)]: Done 1234 tasks      | elapsed: 59.5min
[Parallel(n_jobs=-1)]: Done 1235 tasks      | elapsed: 59.5min
[Parallel(n_jobs=-1)]: Done 1236 tasks      | elapsed: 59.5min
[Parallel(n_jobs=-1)]: Done 1237 tasks      | elapsed: 59.5min
[Parallel(n_jobs=-1)]: Done 1238 tasks      | elapsed: 59.6min
[Parallel(n_jobs=-1)]: Done 1239 tasks      | elapsed: 59.6min
[Parallel(n_jobs=-1)]: Done 1240 tasks      | elapsed: 59.7min
[Parallel(n_jobs=-1)]: Done 1241 tasks      | elapsed: 59.7min
[Parallel(n_jobs=-1)]: Done 1242 tasks      | elapsed: 59.8min
[Parallel(n_jobs=-1)]: Done 1243 tasks      | elapsed: 59.8min
[Parallel(n_jobs=-1)]: Done 1244 tasks      | elapsed: 59.8min
[Parallel(n_jobs=-1)]: Done 1245 tasks      | elapsed: 

[Parallel(n_jobs=-1)]: Done 1361 tasks      | elapsed: 65.6min
[Parallel(n_jobs=-1)]: Done 1362 tasks      | elapsed: 65.6min
[Parallel(n_jobs=-1)]: Done 1363 tasks      | elapsed: 65.6min
[Parallel(n_jobs=-1)]: Done 1364 tasks      | elapsed: 65.7min
[Parallel(n_jobs=-1)]: Done 1365 tasks      | elapsed: 65.7min
[Parallel(n_jobs=-1)]: Done 1366 tasks      | elapsed: 65.8min
[Parallel(n_jobs=-1)]: Done 1367 tasks      | elapsed: 65.8min
[Parallel(n_jobs=-1)]: Done 1368 tasks      | elapsed: 65.9min
[Parallel(n_jobs=-1)]: Done 1369 tasks      | elapsed: 65.9min
[Parallel(n_jobs=-1)]: Done 1370 tasks      | elapsed: 65.9min
[Parallel(n_jobs=-1)]: Done 1371 tasks      | elapsed: 65.9min
[Parallel(n_jobs=-1)]: Done 1372 tasks      | elapsed: 66.0min
[Parallel(n_jobs=-1)]: Done 1373 tasks      | elapsed: 66.0min
[Parallel(n_jobs=-1)]: Done 1374 tasks      | elapsed: 66.1min
[Parallel(n_jobs=-1)]: Done 1375 tasks      | elapsed: 66.1min
[Parallel(n_jobs=-1)]: Done 1376 tasks      | elapsed: 

CPU times: user 14.1 s, sys: 2.13 s, total: 16.3 s
Wall time: 1h 8min 44s


[Parallel(n_jobs=-1)]: Done 1435 out of 1435 | elapsed: 68.7min finished


In [20]:
'''
%%time
resu2=[]
if __name__ == '__main__':
    resu2 = Parallel(n_jobs=2, verbose=20,require='sharedmem')(delayed(solve_out_2)(mparam,Cm,ni,dx,dy,psieq,nseq,pseq,psitt,ns,ps,g,40) for g in G2)
'''

"\n%%time\nresu2=[]\nif __name__ == '__main__':\n    resu2 = Parallel(n_jobs=2, verbose=20,require='sharedmem')(delayed(solve_out_2)(mparam,Cm,ni,dx,dy,psieq,nseq,pseq,psitt,ns,ps,g,40) for g in G2)\n"

In [21]:
'''
%%time
resu3=[]
if __name__ == '__main__':
    resu3 = Parallel(n_jobs=2, verbose=20,require='sharedmem')(delayed(solve_out_2)(mparam,Cm,ni,dx,dy,psieq,nseq,pseq,psitt,ns,ps,g,40) for g in G3)
'''

"\n%%time\nresu3=[]\nif __name__ == '__main__':\n    resu3 = Parallel(n_jobs=2, verbose=20,require='sharedmem')(delayed(solve_out_2)(mparam,Cm,ni,dx,dy,psieq,nseq,pseq,psitt,ns,ps,g,40) for g in G3)\n"

In [29]:

Current=np.zeros((m1))
CurrentN=np.zeros((m1))
Currentm=np.zeros((m1))
CurrentP=np.zeros((m1))
Current=np.zeros((m1))
CurrentN=np.zeros((m1))
Currentm=np.zeros((m1))
CurrentP=np.zeros((m1))
for i in range(0,m1):
    Current[i]=resu[i][0]
    CurrentN[i]=resu[i][1]
    Currentm[i]=resu[i][11]*(consp.k*T*N_0*mu_0)
    
    
'''
for i,num in enumerate(r1[0]):
    Currentm[num]=resu[i][11]

for i,num in enumerate(r2[0]):
    Currentm[num]=resu2[i][11]
for i,num in enumerate(r3[0]):
    Currentm[num]=resu3[i][11]
'''

XBIC=np.zeros((npy,npx))
XBIC[J,I]=Currentm
#mail=np.flipud(mail)
fig, (axes1,axes,axes2) = plt.subplots(figsize=(10, 7),ncols=1,nrows=3)
#zz=axes.pcolor(x*1e6,y*1e6, XBIC)
#cbar = fig.colorbar(zz, ax=axes)
Domain=axes1.imshow((mail[5:20,:]),cmap='RdBu',vmin=-3, vmax=3)
xbiE=axes.imshow((ImXBIC[56:71,1:-1]),cmap='jet')
XBIC=axes2.imshow((XBIC[5:20,:]),cmap='jet',vmin=0, vmax=np.max(Currentm))
fig.colorbar(xbiE, ax=axes, orientation='horizontal')
colo=fig.colorbar(Domain, ax=axes1, orientation='horizontal')
fig.colorbar(XBIC, ax=axes2, orientation='horizontal')
axes1.set_title('Domain')
axes.set_title('XBIC Experiment')
axes2.set_title('XBIC simulation')

plt.show()


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [27]:

%%time
resu=[]
resu = solve_out_2(mparam,Cm,ni,dx,dy,psieq,nseq,pseq,psitt,ns,ps,G[120],570)


/opt/conda/envs/python3/lib/python3.7/site-packages/ipykernel_launcher.py:3: RuntimeWarning: invalid value encountered in true_divide
  This is separate from the ipykernel package so we can avoid doing imports until
/opt/conda/envs/python3/lib/python3.7/site-packages/ipykernel_launcher.py:8: RuntimeWarning: invalid value encountered in true_divide
  


1 229.47086700423097
2 0.8743673572994055
3 0.829813442887172
4 0.7633608879539509
5 0.6693397030736321
6 0.5438119728301274
7 0.4081466095854296
8 0.2692411300415358
9 0.15047715095340664
10 0.07793820385353456
11 0.03586508355351725
12 0.015593468047654824
13 0.01054044171797841
14 0.008115241433415576
15 0.006238979999324101
16 0.004793767870211241
17 0.00368344085273975
18 0.002831610892144454
19 0.002178534402474952
20 0.0016779133899547105
21 0.0013091224680010782
22 0.001158931424072307
23 0.0010292457473042305
24 0.000913269726481758
25 0.0008096915928206272
26 0.000717306621879743
27 0.0006350081749246783
28 0.0005617814412630651
29 0.0004966981823031347
30 0.00043891180041555344
31 0.00038765246006278176
32 0.0003426013278442497
33 0.000303131060746864
34 0.00026803112351103664
35 0.00023685455647637355
36 0.00020919230661320398
37 0.0001846717353544136
38 0.00016295474446792468
39 0.00014373566048675457
40 0.0001267389855286183
41 0.00011171710198709661
42 9.844799248526215e

In [24]:

dirName = 'paper10'
try:
    # Create target Directory
    os.mkdir(dirName)
    print("Directory " , dirName ,  " Created ") 
except FileExistsError:
    print("Directory " , dirName ,  " already exists")


Directory  paper10  Created 


In [25]:

parameterslist=['Nc','Nv','Eg','affinity','mu_e','mu_h','tau_e','tau_h','Sn','Sp','work f','V']
completeName = completeName = (os.getcwd()+"/"+dirName+"/parameters.txt")
textf= open(completeName,"w+")
textf.write('parameters'+' '*10+'CZTS'+' '*12+'CdS'+' '*12+'ZnO'+'\n')
for key in ( parameterslist):
    textf.write(key+' '*(len('parameters')+10-len(key))+"{:.2e}".format(mt1[key])+
                ' '*(len('CZTS')+12-8)+"{:.2e}".format(mt2[key])+
                ' '*(len('CdS')+12-8)+"{:.2e}".format(mt3[key])+
                '\n'*2)
textf.close()


In [26]:
np.save(os.getcwd()+"/"+dirName+"/"+dirName+"_XBIC_S.npy",Currentm)

In [61]:
#np.save('XBICS_shB_G_profile_time_e',Currentm)

In [24]:
#Domain
#XBIC simulation

In [55]:
#array = np.zeros([npy, npx, 4], dtype=np.uint8)
#array[:,:]=[255,255,255,0]
#array[np.where(mail==3)]=[31, 78, 121,255]
#array[np.where(mail==2)]=[91, 155, 213,255]
#array[np.where(mail==1)]=[189, 215, 238,255]

In [17]:
#fig, axes = plt.subplots(figsize=(10, 10))
#plt.imshow((array2),cmap='RdBu',vmin=-3,vmax=3)
#plt.imshow((Z),)
#plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [18]:
#from PIL import Image
#import matplotlib.image as mpimg 
#img = Image.fromarray(array[6:20])
#img.save('doman_G_sh_n.png')
#img = mpimg.imread('doman_G_sh_n.png') 


NameError: name 'array' is not defined

In [12]:
#import imageio
#array2=imageio.imread('doman_G_sh_n.png')


In [15]:
#array2.shape
#139-14
#arrayex = np.zeros([125, npx, 4], dtype=np.uint8)
#arrayex[:,:]=[255,255,255,0]
#array2=np.vstack((arrayex,array2))

In [20]:
#img2 = Image.fromarray(array2)
#img2.save('doman_G_sh_n.png')
#img = mpimg.imread('doman_G_sh_n.png')


In [18]:
#Z=np.zeros((npy,npx))
#Z[J,I]=resu[131][3]
#masked = np.ma.masked_where(mail == 0, mail)
#Z=Z[6:20, :]
#Z1 = 0*np.min(Z)*np.ones(Z.shape)
#masked.data[masked.data!=0]=Z[masked.data!=0]
#xpts=np.linspace(-0.5,npx-1-0.5,Z.shape[1])
#ypts=np.linspace(6.5,20-0.5,Z.shape[0])
#xpts1=np.linspace(-0.5,npx-1-0.5,Z.shape[1])
#ypts1=np.linspace(5.5,20-1-0.5,Z.shape[0])
#X,Y= np.meshgrid(xpts, ypts)
#X1,Y1= np.meshgrid(xpts1, ypts1)
#Z1.shape,xpts1,ypts1,xpts,ypts

In [80]:
'''
fig = plt.figure(figsize=plt.figaspect(0.5)*1.5) #Adjusts the aspect ratio and enlarges the figure (text does not enlarge)
ax = fig.gca(projection='3d')
ax.plot_surface(X1, Y1, Z1 ,rstride=1, cstride=1, facecolors=img, shade=False )
ax.plot_surface(X, Y, Z,cmap='seismic',vmin=-np.max(Z),vmax=np.max(Z),alpha=0.8  )



#ax.plot_surface(X, Y, -np.max(Z), zdir='z',  cmap=cm.ocean)
ax.xaxis.pane.fill = False
ax.yaxis.pane.fill = False
ax.zaxis.pane.fill = False
#ax.zaxis.pane.set_edgecolor('white')
ax.grid(False)
ax.set_zlim(np.min(Z), np.max(Z))
plt.xlabel('x')
plt.ylabel('y')
#ax.w_zaxis.line.set_lw(0.)
ax.set_zticks([])
plt.show()
'''

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [81]:
datan=np.zeros_like(mail)
datan[J,I]=resu[2]

ValueError: setting an array element with a sequence.

In [173]:
masked1 = np.ma.masked_where(mail == 0, mail)



masked1.data[masked1.data!=0]=datan[masked1.data!=0]
plt.figure(figsize=(10, 10));plt.imshow(masked1,cmap='jet')  

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [174]:
abed=[]
for i in range(139):
    abed.append(np.where((pos[:,1]==i) & (mtype[:]==1)))

In [175]:
abed

[(array([ 455,  594,  733,  872, 1011, 1150, 1289]),),
 (array([ 456,  595,  734,  873, 1012, 1151, 1290]),),
 (array([ 457,  596,  735,  874, 1013, 1152, 1291]),),
 (array([ 458,  597,  736,  875, 1014, 1153, 1292]),),
 (array([ 459,  598,  737,  876, 1015, 1154, 1293]),),
 (array([ 460,  599,  738,  877, 1016, 1155, 1294]),),
 (array([ 461,  600,  739,  878, 1017, 1156]),),
 (array([ 323,  462,  601,  740,  879, 1018, 1157, 1295]),),
 (array([ 463,  602,  741,  880, 1019, 1158, 1296]),),
 (array([ 325,  464,  603,  742,  881, 1020, 1159, 1297]),),
 (array([ 326,  465,  604,  743,  882, 1021, 1160]),),
 (array([ 327,  466,  605,  744,  883, 1022, 1161, 1298]),),
 (array([ 328,  467,  606,  745,  884, 1023, 1162, 1299]),),
 (array([ 329,  468,  607,  746,  885, 1024, 1163, 1300]),),
 (array([ 469,  608,  747,  886, 1025, 1164, 1301]),),
 (array([ 470,  609,  748,  887, 1026, 1165, 1302]),),
 (array([ 332,  471,  610,  749,  888, 1027, 1166, 1303]),),
 (array([ 333,  472,  611,  750,  8

In [176]:
range(len(abed))

range(0, 139)

In [177]:
Bs=np.zeros((len(abed)))
for i in range(len(abed)):
    Bs[i]=abed[i][0][-1]

In [178]:
Bs

array([1289., 1290., 1291., 1292., 1293., 1294., 1156., 1295., 1296.,
       1297., 1160., 1298., 1299., 1300., 1301., 1302., 1303., 1304.,
       1168., 1305., 1306., 1307., 1308., 1309., 1424., 1311., 1312.,
       1313., 1314., 1315., 1316., 1181., 1317., 1318., 1319., 1320.,
       1321., 1322., 1323., 1324., 1325., 1326., 1327., 1328., 1329.,
       1330., 1331., 1332., 1333., 1334., 1335., 1336., 1337., 1425.,
       1339., 1340., 1341., 1342., 1343., 1344., 1345., 1346., 1347.,
       1348., 1349., 1350., 1351., 1352., 1353., 1354., 1355., 1356.,
       1357., 1358., 1359., 1360., 1361., 1362., 1363., 1364., 1365.,
       1426., 1367., 1368., 1369., 1370., 1371., 1427., 1373., 1374.,
       1375., 1376., 1377., 1378., 1379., 1380., 1381., 1382., 1383.,
       1384., 1385., 1386., 1387., 1388., 1389., 1390., 1391., 1392.,
       1393., 1394., 1395., 1396., 1397., 1398., 1399., 1400., 1401.,
       1402., 1403., 1428., 1405., 1429., 1407., 1408., 1409., 1410.,
       1411., 1412.,

In [179]:
resu[14]

array([8.60334682e-05, 1.37032198e-04, 1.45651241e-04, ...,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00])

In [180]:
Neig[1433]

array([1433, 1433, 1422, 1434])

In [181]:
Bw=[]
Be=[]
for i in Bs:
    if Neig[int(i),1]==i:
            Bw.append(i)
    if Neig[int(i),3]==i:
            Be.append(i)
        

In [182]:
Bw

[1295.0,
 1298.0,
 1305.0,
 1424.0,
 1317.0,
 1425.0,
 1426.0,
 1427.0,
 1428.0,
 1429.0,
 1430.0,
 1432.0,
 1433.0]

In [183]:
Be

[1294.0,
 1297.0,
 1304.0,
 1424.0,
 1316.0,
 1425.0,
 1426.0,
 1427.0,
 1428.0,
 1429.0,
 1431.0,
 1432.0,
 1434.0]

In [184]:
psic=resu[2]
nsc=resu[3]
psc=resu[4]

In [185]:
VL=0
jnyBs=[]
jpyBs=[]
Bs=Bs.astype("int")
for i in Bs:
    jnyBs.append(-mparam[10,i]*(nsc[i]-nseq[i]))
    jpyBs.append(mparam[11,i]*(psc[i]-pseq[i]))

In [186]:
VU=0
jnxBe=[]
jpxBe=[]
Be=np.array(Be).astype("int")
for i in Be:
    jnxBe.append(mparam[10,i]*(nsc[i]-nseq[i]))
    jpxBe.append(-mparam[11,i]*(psc[i]-pseq[i]))

In [187]:
jnxBw=[]
jpxBw=[]
Bw=np.array(Bw).astype("int")
for i in Bw:
    jnxBw.append(-mparam[10,i]*(nsc[i]-nseq[i]))
    jpxBw.append(mparam[11,i]*(psc[i]-pseq[i]))

In [188]:
Jys=np.array(jnyBs)+np.array(jpyBs)
Jxe=np.array(jnxBe)+np.array(jpxBe)
Jxw=np.array(jnxBw)+np.array(jpxBw)

In [195]:
JtL=(np.sum(Jys)+np.sum(Jxw)-np.sum(Jxe))*dx

In [196]:
JtL

0.6604173194026705

In [197]:
resu[11]

0.6604175975831301

In [198]:
nsc[1434]-nseq[1434]

1.2048823856692224e-08